# Python Notebook to convert H5 file to graph
First script to run for the GNN analysis.
Takes an ".h5" trajectory file from "1_h5_trajectories" as input, and outputs a graph, saved in "2_raw_graphs"

In [1]:
##IMPORTS
import mdtraj as md
import openmm as mm

import mendeleev as me
from mendeleev.fetch import fetch_table
import re
from itertools import combinations
import numpy as np

import torch
from torch_geometric.data import Data

import time

In [2]:
res = np.load("shooting_results.npz")
np.expand_dims(res["shooting_results"][0], axis=0)

array([[279., 221.]])

In [3]:
def convert_traj_to_graph(traj_loc, res_loc, save_loc):
    """A function to convert .h5 mdtraj trajectories to graphs.
    Outputs a graph for each frame of the trajectory.
    Uses the Mendeleev package for atomic features:
    https://mendeleev.readthedocs.io/
    """
    # traj_loc: path to MDTRAJ ".h5" file, non-optional
    # save_loc: path to location to save the resulting graph
    
    print("Start conversion")
    total_time=time.time()
    
    #--Load Trajectory
    traj = md.load_hdf5(fname_h5)
    #--Load Targets
    res = np.load(res_loc)
    targ = res["shooting_results"]#[:,1]/500
    
    #--Get length of trajectory
    traj_len = len(traj)
    #--Get xyz coordinates from trajectory
    traj_xyz = traj.xyz
    #--Get atoms from trajectory
    traj_atoms = [i for i in traj.topology.atoms]
    #--Get atom residue names
    traj_residue = [str(i.residue) for i in traj.topology.atoms]
    #--Get atom symbols via python regex (symbols needed for mendeleev or dictionary)
    traj_atoms_sym = [re.findall(r'[A-Za-z]+|\d+',a.name)[0] for a in traj_atoms]
    
    #--Get all unique atom types in trajectory
    unique_atoms = list(np.unique(traj_atoms_sym))
    
    #--Load Mendeleev fetch table where data is stored
    me_table = fetch_table("elements")
    #--Define the atomic features you want as node features ##CURRENTLY DISUSED!
    atomic_feature_names = ["symbol",
                            "atomic_number",
                            "atomic_radius", #mb rm
                            "atomic_volume", #mb rm
                            #"atomic_weight",
                            #"electron_affinity", #mb rm
                            #"vdw_radius",
                            #"dipole_polarizability", #mb rm
                            #"en_pauling" #mb rm
                            ]
    
    #--Set up dictionary for edge feature bound or unbound
    #keys: B,U (bound, unbound)
    #values: [1,0], [0,1] (one-hot encoding of b/u)
    edge_dict = {"B": [1, 0], "U": [0, 1]}
    
    #--Set up dictionary for epsilon and sigma parameters from OpenMM Forcefield
    #dictionary: [sigma, epsilon]
    #https://github.com/openmm/openmmforcefields/blob/ddea4f61d508f0cc06ad09dd29d0943721890038/amber/ffxml/tip3p_standard.xml#L261
    node_feature_dict = {"Li": [0.18263423721876956, 0.1171084864], 
                         "Cl": [0.4477656957373345, 0.14891274399999999], 
                         "H": [1, 0], 
                         "O": [0.31507524065751241, 0.635968]}
    
    
    
    #--Set up NODE tensor for the graph, 
    #only the xyz coordinates change in each frame, left zero here
    basic_nodes=[]
    for at in traj_atoms_sym:
        #nf = me_table[me_table["symbol"]==at][atomic_feature_names[1:]].values.tolist()[0]+[0,0,0]
        nf = node_feature_dict[at]+[0,0,0]
        basic_nodes.append(nf)
    basic_node_attributes = torch.tensor(basic_nodes, dtype=torch.float)
        
        
    #--Set up edge index tensor
    sender_edges = [list(i) for i in combinations(range(traj.n_atoms),2)]
    receiver_edges = [list(i)[::-1] for i in combinations(range(traj.n_atoms),2)]
    
    edge_index = np.array(sender_edges + receiver_edges)
    
    #--Set up EDGE_ATTR tensor for the graph
    #only the distance changes in each frame
    #here we set up the connection type feature
    start=time.time()
    basic_edge_attributes = torch.tensor([edge_dict["U"] if np.unique(np.array(traj_residue)[edge]).shape==(2,) else edge_dict["B"] for edge in edge_index], dtype=torch.float)
    print("basic_edge_attributes caclulated in {}s".format(np.around(time.time()-start,2)))
    basic_edge_attributes = torch.cat([basic_edge_attributes, torch.full((basic_edge_attributes.shape[0],1),fill_value = 0)], dim=1)
    #Make the edge_index a tensor
    edge_index_tensor = torch.tensor(edge_index.T, dtype=torch.long)
    
    #Timestep here
    for timestep in range(traj_len):
        print("Frame {}/{} converted".format(timestep+1,traj_len))
        #Extract XYZ data from one trajectory frame
        xyz_array = traj[timestep].xyz[0]
        #Calculate distances between nodes as edge attribute
        dist = np.sum(np.square(np.abs(xyz_array[edge_index][:,0,:]-xyz_array[edge_index][:,1,:])),axis=1)
        
        #Make a clone of basic nodes and edge attributes
        g_node_attributes = basic_node_attributes.clone().detach()
        g_edge_attributes = basic_edge_attributes.clone().detach()
    
        #Paste into it the values that change for each frame
        g_node_attributes[:,-3:] = torch.tensor(xyz_array,dtype=torch.float)
        g_edge_attributes[:,-1] = torch.tensor(dist)
        
        #Make tensor of target
        targ_tensor = torch.tensor(np.expand_dims(targ[timestep],axis=0), dtype=torch.float)
        
        #Make a graph of the frame
        graph_of_frame = Data(x = g_node_attributes,
                              edge_index = edge_index_tensor,
                              edge_attr = g_edge_attributes,
                              y = targ_tensor)
        #Save the graph
        torch.save(graph_of_frame,save_loc+"graph_f"+str(timestep)+".pt")
    
    print("\n Total run time: {}m".format(np.around((time.time()-total_time)/60,2)))

In [4]:
fname_h5 = "shooting_points.h5"
fname_npz = "shooting_results.npz"

traj_loc = "aimmd_data/committor_data/"+fname_h5
res_loc = "aimmd_data/committor_data/"+fname_npz
save_loc = "2_raw_graphs/"
#Make graphs
convert_traj_to_graph(traj_loc, res_loc, save_loc)

Start conversion
basic_edge_attributes caclulated in 224.73s
Frame 1/1000 converted
Frame 2/1000 converted
Frame 3/1000 converted
Frame 4/1000 converted
Frame 5/1000 converted
Frame 6/1000 converted
Frame 7/1000 converted
Frame 8/1000 converted
Frame 9/1000 converted
Frame 10/1000 converted
Frame 11/1000 converted
Frame 12/1000 converted
Frame 13/1000 converted
Frame 14/1000 converted
Frame 15/1000 converted
Frame 16/1000 converted
Frame 17/1000 converted
Frame 18/1000 converted
Frame 19/1000 converted
Frame 20/1000 converted
Frame 21/1000 converted
Frame 22/1000 converted
Frame 23/1000 converted
Frame 24/1000 converted
Frame 25/1000 converted
Frame 26/1000 converted
Frame 27/1000 converted
Frame 28/1000 converted
Frame 29/1000 converted
Frame 30/1000 converted
Frame 31/1000 converted
Frame 32/1000 converted
Frame 33/1000 converted
Frame 34/1000 converted
Frame 35/1000 converted
Frame 36/1000 converted
Frame 37/1000 converted
Frame 38/1000 converted
Frame 39/1000 converted
Frame 40/100

Frame 331/1000 converted
Frame 332/1000 converted
Frame 333/1000 converted
Frame 334/1000 converted
Frame 335/1000 converted
Frame 336/1000 converted
Frame 337/1000 converted
Frame 338/1000 converted
Frame 339/1000 converted
Frame 340/1000 converted
Frame 341/1000 converted
Frame 342/1000 converted
Frame 343/1000 converted
Frame 344/1000 converted
Frame 345/1000 converted
Frame 346/1000 converted
Frame 347/1000 converted
Frame 348/1000 converted
Frame 349/1000 converted
Frame 350/1000 converted
Frame 351/1000 converted
Frame 352/1000 converted
Frame 353/1000 converted
Frame 354/1000 converted
Frame 355/1000 converted
Frame 356/1000 converted
Frame 357/1000 converted
Frame 358/1000 converted
Frame 359/1000 converted
Frame 360/1000 converted
Frame 361/1000 converted
Frame 362/1000 converted
Frame 363/1000 converted
Frame 364/1000 converted
Frame 365/1000 converted
Frame 366/1000 converted
Frame 367/1000 converted
Frame 368/1000 converted
Frame 369/1000 converted
Frame 370/1000 converted


Frame 659/1000 converted
Frame 660/1000 converted
Frame 661/1000 converted
Frame 662/1000 converted
Frame 663/1000 converted
Frame 664/1000 converted
Frame 665/1000 converted
Frame 666/1000 converted
Frame 667/1000 converted
Frame 668/1000 converted
Frame 669/1000 converted
Frame 670/1000 converted
Frame 671/1000 converted
Frame 672/1000 converted
Frame 673/1000 converted
Frame 674/1000 converted
Frame 675/1000 converted
Frame 676/1000 converted
Frame 677/1000 converted
Frame 678/1000 converted
Frame 679/1000 converted
Frame 680/1000 converted
Frame 681/1000 converted
Frame 682/1000 converted
Frame 683/1000 converted
Frame 684/1000 converted
Frame 685/1000 converted
Frame 686/1000 converted
Frame 687/1000 converted
Frame 688/1000 converted
Frame 689/1000 converted
Frame 690/1000 converted
Frame 691/1000 converted
Frame 692/1000 converted
Frame 693/1000 converted
Frame 694/1000 converted
Frame 695/1000 converted
Frame 696/1000 converted
Frame 697/1000 converted
Frame 698/1000 converted


Frame 987/1000 converted
Frame 988/1000 converted
Frame 989/1000 converted
Frame 990/1000 converted
Frame 991/1000 converted
Frame 992/1000 converted
Frame 993/1000 converted
Frame 994/1000 converted
Frame 995/1000 converted
Frame 996/1000 converted
Frame 997/1000 converted
Frame 998/1000 converted
Frame 999/1000 converted
Frame 1000/1000 converted

 Total run time: 14.86m
